# Lab 4: 테이블, 데이터 유형, 데이터 로드 🥋 

👉 - 이 실습에서, 식물 뿌리 깊이 값을 저장할 테이블을 생성하고 `INSERT` 구문을 사용하여 데이터를 삽입합니다. 또한 데이터를 쿼리하기 위해 `LIMIT`과 `SELECT` Star(*)의 변형을 사용하는 방법을 알아봅니다.

먼저 이 실습 전반에 걸쳐 사용할 **컨텍스트 정보**를 가져오갰습니다. 

- **Start** 버튼을 클릭하여 이 노트북을 활성화합니다.

- 아래 Python 셀을 실행합니다.

#### :warning: 이 노트북에서 새 세션이 시작될 때마다, 이후 셀에서 사용할 "변수"를 설정하려면 아레 셀을 다시 실행해야 합니다. :warning:

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
user = session.get_current_user().strip('"')
your_db = user + '_DB'
print('Your current CONTEXT information:')
print('---------------------------------')
print(session)
print('Your current USER is ' + user)

## Root Depth 테이블

### 아래 이미지의 데이터를 기반으로, 우리의 테이블은 어떤 모습일까요? 📓 

![Vegetable details table data (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_root_depth_1.png)

`TEXT`와 같은 데이터 유형에 대한 자세한 내용은, Snowflake의 온라인 설명서를 참조하세요. [여기](https://docs.snowflake.com/ko/sql-reference/data-types-text#data-types-for-text-strings").

`NUMBER`와 같은 데이터 유형에 대한 자세한 내용은, Snowflake의 온라인 설명서를 참조하세요. [여기](https://docs.snowflake.com/ko/sql-reference/data-types-numeric#number"). 

### ROOT_DEPTH 테이블 생성하기 🥋 

아래에서 제공된 코드를 검토하여 생성하려는 테이블의 "구조"를 이해합니다.

SQL 셀을 실행하세요. 모든 것이 잘 진행되면 다음 메시지를 받을 것입니다. 

`Table ROOT_DEPTH successfully created.`

In [ ]:
USE ROLE {{user}}_learner_rl;

CREATE TABLE IF NOT EXISTS {{user}}_GARDEN_PLANTS.VEGGIES.ROOT_DEPTH (
   ROOT_DEPTH_ID number(1), 
   ROOT_DEPTH_CODE text(1), 
   ROOT_DEPTH_NAME text(7), 
   UNIT_OF_MEASURE text(2),
   RANGE_MIN number(2),
   RANGE_MAX number(2)
   );

## 노트북 사이드바의 오브젝트 브라우저에서 방금 생성한 테이블 찾기 🥋 

- 노트북 왼쪽에 있는 **Databases** 오브젝트 브라우저를 엽니다.

- **(animal)_GARDEN_PLANTS** 데이터베이스 아래의 **VEGGIES** 스키마에서 **ROOT_DEPTH** 테이블을 찾습니다.

- **새 탭에서 테이블 세부정보 열기** 아이콘(표시됨)을 클릭하여 새 탭 창에서 해당 화면을 엽니다.

![Vegetable details table (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_locate_root_depth_1.png)

### 테이블을 찾을 수 없다면? 📓

- 노트북 왼쪽의 Snowsight **Databases** 오브젝트 브라우저를 새로고침합니다.

- 브라우저 탭을 새로고침합니다.

- Snowsight에서 사용 중인 역할(왼쪽 하단)을 확인합니다. 위 코드로 테이블을 생성할 때 사용한 역할과 동일한가요?

- 노트북 왼쪽 Snowsight **Databases** 오브젝트 브라우저 상단의 **Search objects** 상자를 사용하여 검색하세요.

### 살수로 잘못된 작업을 했다면? 📓 

- 이름을 잘못 지정했나요? 이름을 변경하세요!

- 다른 역할이 소유하고 있나요? 소유권을 이전하세요!

- 잘못된 데이터베이스나 스키마에 있나요? 위치를 옮기세요!!

예제:

- 이름을 잘못 입력했다면 `ALTER TABLE`로 이름을 변경하세요.

- 잘못된 위치에 테이블이 있다면 `ALTER TABLE`로 테이블을 이동하세요.

💡 **팁**: 아래 예제 코드를 참고하여 필요한 경우 수정, 주석해제, 실행하여 문제를 해결하세요.

💡 **힌트**: `{{user}}` 코드는 자동으로 동물 사용자이름을 쿼리에 삽입하므로 별도의 수정이 필요 없습니다.

In [ ]:
-- sample resolutions

-- ALTER TABLE {{user}}_garden_plants.veggies.rootdeath RENAME TO {{user}}_garden_plants.veggies.root_depth;

-- ALTER TABLE {{user}}_garden_plants.flowers.root_depth RENAME TO {{user}}_garden_plants.veggies.root_depth;

## 테이블 정의 확인하기 🥋 

테이블 정의를 확인할 때 표시되는 SQL 코드가 우리가 실행했던 SQL 코드와 **다르다는 점**을 확인하세요. 이는 Snowflake가 내부적으로 몇 가지 변경을 했기 때문입니다!

- 차이점을 찾아볼 수 있나요?

![ROOT_DEPTH table definition (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_view_root_depth_def_1.png)

## 새로운 테이블에 데이터 한 행 삽입하기 

### 테이블에 데이터 행 삽입하기 📓

테이블을 생성했으니 데이터를 넣고자 합니다. 데이터를 테이블에 삽입하는 방법은 여러 가지가 있지만, 가장 간단한 방법부터 시작하여 점차 다양한 옵션을 살펴볼 것입니다. 간단한 방법에서 점차 벗어나면서 더 효율적이고 효과적인 데이터 로드 방법을 배울 것입니다.

이 워크숍이 끝날 때까지 다음과 같은 데이터 로드 방법을 경험하게 될 것입니다: 

- Worksheet에서 `INSERT` 구문을 사용하기

- Load Data Wizard를 사용하기

- `COPY INTO` 구문 사용하기

### 데이터 미리 보기 옵션 사용하기 🥋  

테이블에 데이터를 로드하기 전에, 현재 테이블에 행이 없다는 점을 먼저 확인합니다. 이 정보를 빠르게 액세스할 수 있는 두 가지 방법이 있습니다:

1. 위 예제에서 열었던 **Table details** 화면에서 **Table Details** 탭을 **Data Preview** 탭으로 전환합니다.

1. 요청받으면, 접근 권한이 있는 웨어하우스를 선택합니다. 

![Preview ROOT_DEPTH data 1 (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_root_depth_zero_rows_1.png)

1. 다른 방법으로, 노트북 옆의 Databases 탭에서 오브젝트 브라우저를 통해 테이블을 찾습니다. 

1. 테이블 이름을 클릭하면 아래에 오브젝트 세부 정보 창이 나타납니다.

1. 돋보기 아이콘을 클릭하여 테이블 데이터를 미리 봅니다.(현재 이 테이블은 비어 있음)

![Preview ROOT_DEPTH data 2 (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_root_depth_zero_rows_2.png)

### 아래 Insert 구문을 사용해서 ROOT_DEPTH 테이블에 한 행 삽입하기 🥋 

1. 아래 SQL 셀에 있는 구문을 실행합니다.

1. **그런 다음**, 위에서 설명한 데이터 미리보기 방법 중 하나를 사용하여 방금 추가한 데이터 행을 확인하세요.

💡 **팁**: 매번 오브젝트의 전체 이름을 입력하는 대신, 코드로 컨텍스트(데이터베이스와 스키마)를 설정할 수 있습니다.

In [ ]:
USE SCHEMA {{user}}_GARDEN_PLANTS.VEGGIES;

INSERT INTO ROOT_DEPTH (
    ROOT_DEPTH_ID,
    ROOT_DEPTH_CODE,
    ROOT_DEPTH_NAME,
    UNIT_OF_MEASURE,
    RANGE_MIN,
    RANGE_MAX
)
VALUES (
    1,
    'S',
    'Shallow',
    'cm',
    30,
    45
);

## 악명 높은 "Does Not Exist or Not Authorized" 오류 ❗ 

쿼리를 실행할 때 종종 다음과 같은 메시지가 표시될 수 있습니다: **Table 'TABLE_NAME' does not exist or not authorized.**


이 오류는 어렵지 않게 수정할 수 있습니다. 역할을 변경하거나, 설정된 컨텍스트를 변경하거나, 오브젝트의 전체 위치를 추가하여 완전한 이름으로 참조하면 됩니다.

때로는 소유권 변경이 해당 항목을 보기 위해 필요합니다. 때로는 이름을 변경하거나 이동해야 할 수도 있습니다. 스스로에게 질문을 해보세요. "내가 이걸 어디에서 찾고 있는가? 이것은 어디에 있는가? 이것의 이름이 맞는가?" 그리고 "이것을 찾는 것은 누구인가? 해당 역할에 접근 권한이 있는가?"

문제를 해결하려면 다음 중 하나를 수행합니다:

1. 컨텍스트 재설정 (`USE DATABASE` 또는 `USE SCHEMA` 명령을 실행하는 것을 고려)

1. 오브젝트를 참조할 때 긴(전체) 이름을 사용하세요. 예:`(animal)_GARDEN_PLANTS.VEGGIES.ROOT_DEPTH` _(데이터베이스.스키마.테이블)_

## Select Star 와 Limit 📓

"`SELECT *`" (기술적으로 asterisk)로 시작하는 Select Star 구문은 "Select Asterisk"라고 말하는 것은 더 길기 때문에 보통 Select Star라고 부릅니다. Select Star는 테이블의 컬럼 이름을 하나씩 나열하지 않고도 모든 컬럼을 가져오는 간단한 방법입니다.

### 새 테이블에서 Select Star 실행하기 🥋

다음 SQL 셀의 샘플 코드를 수정하여 `ROOT_DEPTH` 테이블의 모든 레코드를 Select 하는 쿼리를 작성해 보세요.

- 아래 구문에서 해시 (`#`) 문자를 적절하게 변경하세요.

In [ ]:
-- modify this sample code
SELECT * FROM ROOT_DEPTH;

### Select star 변형 📓

Select Star로 작업할 때, Snowflake는 입력을 최소화하면서 출력을 원하는 대로 조정하는데 도움이 될 수 있는 몇가지 SQL-sugar 옵션을 제공합니다. 이는 컬럼이 많은 테이블, 테이블 함수, 뷰와 작업할 때 매우 유용합니다. 쿼리를 위해 수십, 수백 개의 컬럼 이름을 수동으로 입력하는 것은 비효율적이므로, 시간을 더 잘 활용할 방법을 제공하는 옵션입니다!

`SELECT *`를 지정할 때, 다음 옵션을 포함할 수 있습니다: `EXCLUDE`, `REPLACE`, `RENAME`, 그리고 `ILIKE`

- `EXCLUDE`: 지정된 컬럼을 제외하고 모든 컬럼을 반환합니다.
- `REPLACE`: 모든 컬럼을 반환하며, 지정된 컬럼의 값을 평가된 표현식으로 대체합니다.
- `RENAME`:  모든 컬럼을 반환하며, 결과에 사용할 컬럼 별칭을 지정합니다.
- `ILIKE`:   지정된 패턴과 일치하는 모든 컬럼을 반환합니다.

### 새 테이블에서 아래 예제를 실행하여 이러한 간단한 변형으로 컬럼 출력을 조정해 보세요

In [ ]:
-- return all columns from your table, except those in the exclusion list, using EXCLUDE

SELECT * EXCLUDE (range_min, range_max)
FROM ROOT_DEPTH;

In [ ]:
-- return all columns from your table, and append the String '-RDC' to the values in the root_depth_code column, using REPLACE

SELECT * REPLACE (root_depth_code||'-RDC' AS root_depth_code)
FROM ROOT_DEPTH;

In [ ]:
-- return all columns from your table, and change the title/header for the range columns, using RENAME

SELECT * RENAME (range_min AS root_depth_range_min, 
                 range_max AS root_depth_range_max)
FROM ROOT_DEPTH;

In [ ]:
-- return all columns from your table that contain the string `ROOT`, using ILIKE

SELECT * ILIKE '%ROOT%'
FROM ROOT_DEPTH;

### Select Star 와 Limit 배우기 📓  

모든 컬럼은 보고 싶지만 모든 행을 보고 싶지 않을 때는. `LIMIT`과 함께 Select Star를 실행할 수 있습니다. Limit을 사용하면 적은 수의 행만 가져오도록 제한할 수 있습니다. 이렇게 하면 행이 수백만 개일때, 몇개만 확인하려는 경우 불필요하게 모든 행을 가져와 컴퓨팅 리소스를 낭비하지 않게 됩니다.

물론 지금은 테이블에 행이 한개 밖에 없지만, 이후에는 더 많은 행이 추가될 것입니다. 또한 이후에 더 많은 데이터를 로드한 새 테이블도 생성하게 될 것입니다.

In [ ]:
SELECT *
FROM ROOT_DEPTH
LIMIT 1;

## Lesson 4 챌린지 실습🎯 

### ROOT_DEPTH 테이블에 두 개의 행 추가하기 🎯 

아래 SQL 셀에서 이전에 사용했던 `INSERT` 구문을 편집하고 둘다 실행하여, 두 개의 행을 **ROOT_DEPTH** 테이블에 추가합니다.

- 해시 (`#`) 문자를 아래 이미지에서 확인할 수 있는 값으로 대체하세요.

![ROOT_DEPTH add rows (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_root_depth_add_rows_1.png)

In [ ]:
-- Edit the values (replace the # characters) in the cell to match those in the SECOND row of the image above, then run 

INSERT INTO ROOT_DEPTH (
    ROOT_DEPTH_ID,
    ROOT_DEPTH_CODE,
    ROOT_DEPTH_NAME,
    UNIT_OF_MEASURE,
    RANGE_MIN,
    RANGE_MAX
)
VALUES (
    2,
    'M',
    'Medium',
    'cm',
    45,
    60
);

In [ ]:
-- Edit the values (replace the # characters) in the cell to match those in the THIRD row of the image above, then run

INSERT INTO ROOT_DEPTH (
    ROOT_DEPTH_ID,
    ROOT_DEPTH_CODE,
    ROOT_DEPTH_NAME,
    UNIT_OF_MEASURE,
    RANGE_MIN,
    RANGE_MAX
)
VALUES (
    3,
    'D',
    'Deep',
    'cm',
    30,
    45
);

### 결과 검토 🥋

- `SELECT *`를 실행하여 **ROOT_DEPTH** 테이블에 모두 3 개의 행이 있는지 확인하세요. 

- 위 이미지와 결과를 비교하세요.

In [ ]:
SELECT *
FROM ROOT_DEPTH

## SQL이 처음이신가요? 도움이 필요하신가요? 아래 코드 샘플을 확인해 보세요 📓

아래는 SQL을 사용해 특정 작업을 수행하는 방법을 보여주는 예제입니다. 필요에 맞게 편집하지 않고 그대로 실행하지는 마십시오.

### Example one - 한 번에 여러 행 추가하기

    INSERT INTO root_depth (
    
        root_depth_id, root_depth_code, root_depth_name, unit_of_measure, range_min, range_max)  

    VALUES
    
        (5,'X','short','in',66,77),
    
        (8,'Y','tall','cm',98,99);

    
### Example two - 테이블에서 필요하지 않은 행 삭제하기

    DELETE FROM root_depth

    WHERE root_depth_id = 9;

### Example three - 특정 행의 컬럼 값을 변경하기

    UPDATE root_depth

        SET root_depth_id = 7

    WHERE root_depth_id = 9;

### Example four - 테이블의 모든 행을 삭제하고 새로 시작하기

    TRUNCATE TABLE root_depth;

## Lab 4 검토 🏁 

### Lab 5 준비가 되셨나요?

- **(animal)_GARDEN_PLANTS** 데이터베이스에 4개의 스키마가 있나요? 

- **VEGGIES** 스키마에 **ROOT_DEPTH**라는 단일 테이블이 있고, 그 테이블에 3개의 행이 있나요?

이 모든 질문에 **예**라고 대답할 수 있다면 다음 단계로 진행하세요. 그렇지 않다면, 잘못된 부분을 수정하고 다시 확인하세요!

## 지식 테스트 🔎

아래 Python 셀을 실행하여 Streamlit 기반의 위젯을 표시하고 Snowflake 특징과 기능에 대한 질문에 답하세요. 지금은 이 코드가 무엇을 하는지 이해할 필요가 없습니다. 그냥 코드를 실행하세요.

질문에 올바르게 답해야 다음 섹션으로 진행할 수 있습니다.

In [ ]:
import streamlit as st
st.divider()
question = "Snowflake가 테이블을 생성할 때 SQL에 약간의 변경을 가했습니다. 어떤 변경이 이루어졌습니까?"
options = ["아래 보기 중 고르세요...",
           "A) Snowflake가 테이블 이름을 변경했습니다.", 
           "B) Snowflake가 NUMBER 데이터 유형을 VARCHAR로 변환했습니다.", 
           "C) Snowflake가 TEXT 데이터 유형을 VARCHAR로 변환했습니다.",
           "D) Snowflake가 소유자를 SYSADMIN으로 변경했습니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '789a492d8fbb0cab5d05673117e56326'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "노트북에서 테이블의 데이터를 어떻게 미리 볼 수 있습니까?"
options = ["아래 보기 중 고르세요...",
           "A) 오브젝트 선택기에서 테이블을 찾은 후 테이블 이름을 클릭하고, 돋보기 아이콘을 클릭합니다.", 
           "B) 노트북 컨텍스트의 TABLE 드롭다운 메뉴를 PREVIEW로 설정합니다. 이는 데이터 미리보기를 위해 설계되었습니다.", 
           "C) 노트북 컨텍스트의 ROLE 드롭다운 메뉴를 PUBLIC으로 설정합니다. 이는 데이터 미리보기를 위해 설계되었습니다.",
           "D) 테이블 이름을 마우스 오른쪽 버튼으로 클릭한 후 \"Table Preview Now\" 옵션을 선택합니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '924f28ec9c53c5b2badabfa308999bc3'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "SELECT 구문에서 LIMIT의 역할은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) 제한된 행 집합을 빠르게 요청하는 데 사용할 수 있습니다.", 
           "B) 제한된 테이블 집합을 빠르게 요청하는 데 사용할 수 있습니다.", 
           "C) 제한된 컬럼 집합을 빠르게 요청하는 데 사용할 수 있습니다.",
           "D) 제한된 스키마 집합을 빠르게 요청하는 데 사용할 수 있습니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '48886ffd3988718c32175f5c675fe4d7'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "이전에 생성된 ROOT_DEPTH 테이블의 소유자는 어떤 역할이어야 합니까?"
options = ["아래 보기 중 고르세요...",
           "A) ACCOUNTADMIN", 
           "B) SYSADMIN", 
           "C) PUBLIC",
           "D) (animal)_LEARNER_RL"]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = 'd249e599063c3525e1566593f7b2ffb3'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

## 다음 단계

실습 단계를 완료하고 **지식 테스트** 질문에 올바르게 답했다면, Snowflake 강사의 안내에 따라 다음 노트북으로 진행하세요.